In [1]:
import numpy as np
import cv2 
import os
import imutils
import matplotlib.pyplot as plt
from PIL import Image
"""
This function is used to crop the images contained in the path directory and save it in the dest directory
It also displays the cropped images
"""
def crop(path,dest):
    if os.path.exists(dest):
        os.system('rm -r dest')
    if not os.path.exists(dest):
        os.makedirs(dest)
    if not os.path.exists(path):
        os.makedirs(path)
    process(path, dest)
    show_images(dest)
    return

"""
This function is used to display the images contained in the directory
"""
def show_images(directory):
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            print(f)
            if f.endswith('.ppm') or f.endswith('.jpeg') or f.endswith('.jp2'):
                cv2.imshow('image', cv2.imread(f))
                cv2.waitKey()
    return

"""
This function is used to process the images contained in the path directory and save it in the dest directory
"""
def process(path, dest):
    directory = path
    i = 0
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        if not os.path.isfile(f):
            print("Expected the directory to contain only files") 
        if os.path.isfile(f):
            if filename.endswith('.ppm') or filename.endswith('.jpeg') or filename.endswith('.jp2'):
                print(f)
                raw_image = cv2.imread(f) #load the image
                process_image = raw_image.copy() #copy the image
                #convert the image to grayscale
                gray = cv2.cvtColor(process_image, cv2.COLOR_BGR2GRAY)
                #apply a Gaussian blur 
                bilateral_filtered_image = cv2.GaussianBlur(gray,(3,3),0)
                #apply a Canny edge detector
                edge_detected_image = cv2.Canny(bilateral_filtered_image, 90, 170)

                #Find the contours in the image
                contours = cv2.findContours(edge_detected_image.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                contours = imutils.grab_contours(contours)
                contours = sorted(contours, key=cv2.contourArea, reverse=True)[:8]

                triangles = {}
                squares = {}
                octagons = {}
                circles = {}

                #loop over the contours that were found in the image 
                for i, contour in enumerate(contours): 
                    length = cv2.arcLength(contour,True)
                    approx = cv2.approxPolyDP(contour,0.01*length, True)
                    if not cv2.isContourConvex(approx):
                        continue

                    #Checks if the shape defined by the contour is "regular" in terms of its edges
                    area = cv2.contourArea(contour)
                    perimeter = cv2.arcLength(contour, True)
                    circularity = 4 * np.pi * area / perimeter ** 2
                    if circularity < 0.8:  
                        continue
                    
                    #The contour is a triangle
                    if len(approx) == 3:
                        triangles[i] = cv2.contourArea(contour)
                    
                    #The contour is a square or a non-flat rectangle
                    elif len(approx) == 4:
                        (x, y, w, h) = cv2.boundingRect(approx)
                        ar = w / float(h)
                        if (ar >= 0.8 and ar <= 1.2):
                            squares[i] = cv2.contourArea(contour)
                        else: # The contour is a flat rectangle (can't be a road sign)
                            continue

                    #The contour is an octagon
                    elif len(approx) == 8:
                        octagons[i] = cv2.contourArea(contour)

                    #The contour is a circle
                    elif len(approx) > 8:
                        (x, y), radius = cv2.minEnclosingCircle(contour)
                        area = cv2.contourArea(contour)
                        if radius > 0 and area / (np.pi * radius**2) >= 0.8:
                            circles[i] = area
                    #The contour is not a shape that corresponds to a road sign
                    else:
                        continue

                #For each shape, find the contour with the largest area
                #Keep at most one contour per shape
                #Unless no contours of that shape were found
                try:
                    largest_tri = max(triangles, key=triangles.get)
                    largest_tri_contour = contours[largest_tri]
                except ValueError:
                    largest_tri_contour = None
                try:
                    largest_sq = max(squares, key=squares.get)
                    largest_sq_contour = contours[largest_sq]
                except ValueError:
                    largest_sq_contour = None
                try:
                    largest_oct = max(octagons, key=octagons.get)
                    largest_oct_contour = contours[largest_oct]
                except ValueError:
                    largest_oct_contour = None
                try:
                    largest_circle = max(circles, key=circles.get)
                    largest_circle_contour = contours[largest_circle]
                except ValueError:
                    largest_circle_contour = None

                # Find contour with largest area overall
                shapes = [c for c in [largest_tri_contour, largest_sq_contour, largest_oct_contour, largest_circle_contour] if c is not None]

                # If no contours were found in the image 
                # The result is not cropped
                if(len(shapes) == 0):
                    cv2.imwrite(dest+'/'+filename, raw_image)
                    print("not cropped")
                else:
                    largest_shape = max(shapes, key=lambda x: cv2.contourArea(x))
                    #Find the bounding rectangle of the contour
                    (x,y,w,h) = cv2.boundingRect(largest_shape)
                    cv2.rectangle(raw_image, (x,y), (x+w,y+h), (0,255,0), 2)
                    #Draw the contour on the image
                    cv2.drawContours(raw_image, [largest_shape],  -1, (255,0,0), 2)
                    cv2.waitKey()
                    #Crop the image to the bounding rectangle
                    raw_image = raw_image[y:y + h, x:x + w]
                    img = cv2.imread(f)
                    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)   
                    #Add the result to the destination directory
                    cv2.imwrite(dest+'/'+filename, raw_image)
    
def main():
    crop('eval_kaggle2', 'result_croppped_eval_2')
    return
if __name__ == "__main__":
    main()

eval_kaggle2\1.ppm
not cropped
eval_kaggle2\10.ppm
not cropped
eval_kaggle2\11.ppm
not cropped
eval_kaggle2\12.ppm
not cropped
eval_kaggle2\13.ppm
not cropped
eval_kaggle2\14.ppm
eval_kaggle2\15.ppm
eval_kaggle2\16.ppm
not cropped
eval_kaggle2\17.ppm
not cropped
eval_kaggle2\18.ppm
eval_kaggle2\19.ppm
not cropped
eval_kaggle2\2.ppm
eval_kaggle2\20.ppm
not cropped
eval_kaggle2\21.ppm
eval_kaggle2\22.ppm
not cropped
eval_kaggle2\23.ppm
not cropped
eval_kaggle2\24.ppm
not cropped
eval_kaggle2\25.ppm
not cropped
eval_kaggle2\26.ppm
not cropped
eval_kaggle2\27.ppm
not cropped
eval_kaggle2\28.ppm
not cropped
eval_kaggle2\29.ppm
not cropped
eval_kaggle2\3.ppm
not cropped
eval_kaggle2\30.ppm
not cropped
eval_kaggle2\31.ppm
not cropped
eval_kaggle2\32.ppm
eval_kaggle2\33.ppm
eval_kaggle2\34.ppm
not cropped
eval_kaggle2\35.ppm
not cropped
eval_kaggle2\36.ppm
eval_kaggle2\37.ppm
not cropped
eval_kaggle2\38.ppm
not cropped
eval_kaggle2\39.ppm
not cropped
eval_kaggle2\4.ppm
not cropped
eval_kaggle2